
# **Rastreamento de objetos simples por cores**

#### **iremos aprender:**
1. Como usar um filtro de cor HSV para criar uma máscara e rastrear um objeto.


In [1]:
# Our Setup, Import Libaries, Create our Imshow Function and Download our Images
import cv2
import numpy as np
from matplotlib import pyplot as plt

# Define our imshow function 
def imshow(title = "Image", image = None, size = 10):
    w, h = image.shape[0], image.shape[1]
    aspect_ratio = w/h
    plt.figure(figsize=(size * aspect_ratio,size))
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.title(title)
    plt.show()

!wget https://moderncomputervision.s3.eu-west-2.amazonaws.com/bmwm4.mp4

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [14]:
# Rastreamento de Objetos
# Importando bibliotecas necessárias para processamento de imagens e manipulação de vídeos
import cv2  # Biblioteca OpenCV para análise de imagens e vídeos
import numpy as np  # Numpy para operações numéricas

# Inicializar câmera ou arquivo de vídeo
# Descomente a linha abaixo para usar a webcam em vez de um arquivo de vídeo
# cap = cv2.VideoCapture(0)

# Definir o intervalo da cor alvo no espaço de cores HSV
# Os limites inferior e superior determinam o intervalo de cores a serem rastreadas
lower = np.array([0, 0, 0])  # Limite inferior HSV para cor preta
upper = np.array([180, 255, 50])  # Limite superior HSV para cor preta

# Criar uma lista vazia para armazenar os pontos centrais dos objetos detectados
# Essa lista é usada para criar o efeito de rastro conectando os pontos
points = []

# Carregar o arquivo de vídeo
# Substitua 'images/bmw2.mp4' pelo caminho do seu arquivo de vídeo
cap = cv2.VideoCapture('images/pessoa.mp4')

# Recuperar a largura e altura dos quadros do vídeo
# Esses valores são necessários para formatar corretamente o vídeo de saída
width = int(cap.get(3))  # Largura do quadro de vídeo
height = int(cap.get(4))  # Altura do quadro de vídeo

# Definir o codec e criar um objeto VideoWriter para salvar o vídeo processado
# A saída será salva como 'bmw2_output.avi'
out = cv2.VideoWriter('pessoa_output-HSVTracking.avi', cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'), 30, (width, height))

# Ler o primeiro quadro do vídeo para extrair suas dimensões
ret, frame = cap.read()
Height, Width = frame.shape[:2]  # Extrair a altura e largura do primeiro quadro
frame_count = 0  # Contador de quadros processados
radius = 0  # Raio do objeto detectado (inicializado como zero)

# Iniciar o processamento dos quadros do vídeo em um loop
while True:
    # Capturar cada quadro do vídeo
    ret, frame = cap.read()
    
    if ret:  # Se um quadro for capturado com sucesso
        # Converter o quadro de BGR para o espaço de cores HSV
        hsv_img = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

        # Criar uma máscara para a cor alvo usando o intervalo HSV definido
        mask = cv2.inRange(hsv_img, lower, upper)

        # Encontrar contornos na máscara
        contours, _ = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Ponto central padrão caso nenhum contorno seja encontrado
        center = (int(Height / 2), int(Width / 2))

        if len(contours) > 0:  # Se pelo menos um contorno for detectado
            # Encontrar o maior contorno com base na área
            c = max(contours, key=cv2.contourArea)

            # Calcular o menor círculo que engloba o maior contorno
            (x, y), radius = cv2.minEnclosingCircle(c)

            # Calcular momentos para encontrar o centróide do contorno
            M = cv2.moments(c)
            try:
                # Calcular o centro do contorno usando os momentos
                center = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))
            except ZeroDivisionError:
                # Tratar divisão por zero caso os momentos sejam inválidos
                center = (int(Height / 2), int(Width / 2))

            # Processar apenas contornos com raio maior que 25 pixels
            if radius > 25:
                # Desenhar um círculo ao redor do objeto detectado
                cv2.circle(frame, (int(x), int(y)), int(radius), (0, 0, 255), 2)

                # Marcar o centróide do objeto detectado
                cv2.circle(frame, center, 5, (0, 255, 0), -1)

                cv2.drawContours(frame, [c], -1, (255,0,0), 3)

            # Adicionar o centro à lista de pontos para criar o rastro
            points.append(center)

        # Se o raio for suficientemente grande, desenhar o rastro
        if radius > 25:
            for i in range(1, len(points)):
                try:
                    # Desenhar linhas conectando os pontos para formar o rastro
                    cv2.line(frame, points[i - 1], points[i], (0, 255, 0), 2)
                except:
                    pass

            # Reiniciar o contador de quadros
            frame_count = 0

        # Escrever o quadro processado no vídeo de saída
        out.write(frame)
    else:
        # Encerrar o loop se nenhum quadro for capturado (fim do vídeo)
        break

# Liberar os objetos de captura e gravação de vídeo
cap.release()
out.release()


In [ ]:
!ffmpeg -i /content/bmwm4_output.avi bmwm4_output.mp4 -y

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

In [ ]:
from IPython.display import HTML
from base64 import b64encode

mp4 = open('bmwm4_output.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

In [ ]:
HTML("""
<video controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

Output hidden; open in https://colab.research.google.com to view.